<a href="https://colab.research.google.com/github/pedroarroteia/SAD-LAB1/blob/main/Damos_lhe_as_boas_vindas_ao_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import random
import os
import re
import copy
import zipfile
import gdown
import shutil
import tensorflow as tf
import cv2
import numpy as np



In [3]:
import os
import tensorflow as tf
import cv2
import numpy as np

class AircraftFuselageDataset(tf.data.Dataset):
    def __new__(cls, img_folder, label_folder, batch_size=32, shuffle=True, img_size=(416, 416)):
        img_files = sorted([os.path.join(img_folder, f) for f in os.listdir(img_folder) if f.endswith('.jpg')])
        label_files = sorted([os.path.join(label_folder, f.replace('.jpg', '.txt')) for f in os.listdir(img_folder) if f.endswith('.jpg')])

        dataset = tf.data.Dataset.from_tensor_slices((img_files, label_files))

        # Map function to load and preprocess the images and labels
        dataset = dataset.map(lambda img, label: cls.process_data(img, label, img_size), num_parallel_calls=tf.data.AUTOTUNE)

        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(img_files))
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

        return dataset

    @staticmethod
    def process_data(img_path, label_path, img_size):
        # Load and preprocess image
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, img_size)

        # Load and preprocess labels (bounding boxes)
        label = tf.io.read_file(label_path)
        label = tf.strings.split(label, '\n')
        label = label.to_tensor()  # Converts it into a tensor of strings

        boxes = []
        for line in label:
            if line != '':  # Ignore empty lines
                parts = tf.strings.split(line).to_tensor()
                label_class = tf.strings.to_number(parts[0], out_type=tf.int32)
                x_center = tf.strings.to_number(parts[1], out_type=tf.float32)
                y_center = tf.strings.to_number(parts[2], out_type=tf.float32)
                width = tf.strings.to_number(parts[3], out_type=tf.float32)
                height = tf.strings.to_number(parts[4], out_type=tf.float32)
                boxes.append([label_class, x_center, y_center, width, height])

        # Convert boxes to a tensor
        boxes = tf.convert_to_tensor(boxes, dtype=tf.float32)

        return img, boxes


# Corrigir o caminho para o formato raw string para evitar problemas com barras
img_folder = r"C:\Users\jpedr\Desktop\FCT\Tese\Database\Aircraft_Fuselage_DET2023\aircraft_fuselage_yolo\images"
label_folder = r"C:\Users\jpedr\Desktop\FCT\Tese\Database\Aircraft_Fuselage_DET2023\aircraft_fuselage_yolo\labels"

# Parâmetros
batch_size = 32

# Criar o dataset
dataset = AircraftFuselageDataset(img_folder, label_folder, batch_size=batch_size)

# Iterar no dataset
for batch in dataset:
    images, boxes = batch
    print(f"Imagens shape: {images.shape}")
    print(f"Boxes: {boxes.shape}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jpedr\\Desktop\\FCT\\Tese\\Database\\Aircraft_Fuselage_DET2023\\aircraft_fuselage_yolo\\images'